In [1]:
import glob
import os 
import numpy as np
import cv2
import mediapipe
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import keras 
import keras_tuner
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Dropout,BatchNormalization
from keras.utils import np_utils
import tensorflow.keras
from keras.applications.vgg16 import VGG16
from sklearn.model_selection import train_test_split

2023-04-18 23:42:57.289573: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
names = ['buildings','forest', 'glacier','mountain', 'sea', 'street']

In [3]:
train_data = []
train_image_names=[]
train_labels = []
test_data = []
test_labels = []
k=0
for i in names:
    print(i)
    os.chdir(r'D:\Data Sets\Intel Image Classification\seg_train\seg_train/'+i+'/')
    file_name = os.listdir()
    train_image_names = train_image_names + file_name
    for j in file_name:
        img=cv2.imread(r'D:\Data Sets\Intel Image Classification\seg_train\seg_train/'+i+'/'+j)
        img = cv2.resize(img,(100,100))
        train_data.append(img)
        train_labels.append(k)
    os.chdir(r'D:\Data Sets\Intel Image Classification\seg_test\seg_testt/'+i+'/')
    file_name = os.listdir()
    for j in file_name:
        img=cv2.imread(r'D:\Data Sets\Intel Image Classification\seg_test\seg_test/'+i+'/'+j)
        img = cv2.resize(img,(100,100))
        test_data.append(img)
        test_labels.append(k) 
    k=k+1

buildings
forest
glacier
mountain
sea
street


In [4]:
train_data = np.array(train_data)
train_labels = np.array(train_labels)
test_data = np.array(test_data)
test_labels = np.array(test_labels)

In [5]:
train_data = train_data/255
test_data = test_data/255

In [6]:
train_label = keras.utils.to_categorical(train_labels,dtype='float32')
test_label = keras.utils.to_categorical(test_labels,dtype='float32')

In [7]:
x_train, x_test, y_train, y_test = train_test_split(train_data,train_label,stratify=train_label,test_size=0.08,random_state=100)

In [8]:
model=Sequential()
model.add(Conv2D(32,(3,3),padding="same",activation="relu",input_shape=(100,100,3)))
model.add(MaxPooling2D(pool_size=(1,1)))

model.add(Conv2D(32,(3,3),padding="same",activation="relu"))
model.add(MaxPooling2D(pool_size=(1,1)))

model.add(Conv2D(64,(3,3),padding="same",activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3,3),padding="same",activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(128,(3,3),padding="same",activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))


model.add(Conv2D(128,(3,3),padding="same",activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(256,(3,3),padding="same",activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))



model.add(Flatten())
model.add(Dense(128,activation="relu",kernel_initializer="he_uniform"))
model.add(Dense(6,activation="softmax"))

2023-04-18 23:43:26.641641: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=['accuracy'])

In [10]:
model.fit(x_train,y_train,batch_size=32,epochs=5,validation_split=0.20)

Epoch 1/5
323/323 [==============================] - 616s 2s/step - loss: 1.1054 - accuracy: 0.5558 - val_loss: 0.8704 - val_accuracy: 0.6671
Epoch 2/5
323/323 [==============================] - 611s 2s/step - loss: 0.7710 - accuracy: 0.7022 - val_loss: 0.7910 - val_accuracy: 0.6872
Epoch 3/5
323/323 [==============================] - 606s 2s/step - loss: 0.6519 - accuracy: 0.7542 - val_loss: 0.6607 - val_accuracy: 0.7511
Epoch 4/5
323/323 [==============================] - 703s 2s/step - loss: 0.5629 - accuracy: 0.7956 - val_loss: 0.6331 - val_accuracy: 0.7635
Epoch 5/5
323/323 [==============================] - 773s 2s/step - loss: 0.4723 - accuracy: 0.8290 - val_loss: 0.5310 - val_accuracy: 0.8173


In [11]:
model.evaluate(x_test,y_test)

36/36 [==============================] - 23s 661ms/step - loss: 0.4910 - accuracy: 0.8237


[0.4909668564796448, 0.8236865401268005]

In [12]:
y_train_pred = model.predict(train_data)
y_train_pred

439/439 [==============================] - 278s 632ms/step


array([[5.0475168e-01, 7.7528574e-02, 8.0796763e-02, 5.5681378e-02,
        7.5622387e-02, 2.0561917e-01],
       [9.9985230e-01, 7.0282204e-07, 8.0657206e-05, 5.3905969e-06,
        4.5933477e-05, 1.4951135e-05],
       [4.7755489e-01, 1.4721316e-01, 9.9836905e-03, 1.0545317e-03,
        2.0114200e-03, 3.6218229e-01],
       ...,
       [3.8451225e-01, 5.9749372e-03, 1.1442290e-02, 1.3058872e-02,
        6.9912239e-03, 5.7802051e-01],
       [3.3076346e-02, 4.3558422e-03, 2.7279451e-03, 6.9653871e-04,
        1.1763326e-03, 9.5796692e-01],
       [4.5380395e-05, 5.1497345e-06, 1.2336746e-07, 1.2829240e-07,
        1.8974468e-08, 9.9994922e-01]], dtype=float32)

In [13]:
y_pred_classes=[np.argmax(i) for i in y_train_pred] 

In [14]:
y_pred_classes

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 5,
 0,
 5,
 0,
 0,
 5,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 5,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 5,
 5,
 0,
 0,
 0,
 5,
 0,
 0,
 0,
 0,
 5,
 0,
 0,
 0,
 0,
 5,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 5,
 0,
 0,
 5,
 0,
 0,
 5,
 0,
 0,
 5,
 5,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 5,
 0,
 0,
 0,
 4,
 5,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 5,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 5,
 5,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 3,
 0,
 0,
 0,
 4,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 5,
 0,
 0,
 0,
 5,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 5,
 5,
 0,
 0,
 0,
 4,
 0,
 0,
 0,
 0,
 4,
 0,
 5,
 0,
 0,
 0,
 0,
 0,
 5,
 0,
 0,
 2,
 0,
 0,
 0,
 5,
 0,
 0,
 0,
 0,
 5,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 5,
 5,
 0,
 0,
 0,
 0,
 0,
 0,
 5,
 0,
 0,
 0,
 5,
 0,
 0,
 0,
 5,
 0,
 1,
 0,
 4,
 5,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [17]:
pd.DataFrame([train_image_names,y_pred_classes],names=['Image_name','Predictied_labels']).T.to_csv(r'D:\Data Sets\Intel Image Classification/cnn1.csv')

In [ ]:
🤞

a = keras_tuner.RandomSearch(models,objective='val_accuracy',max_trials=3,directory='/Users/vamsi/Downloads/Intel Image Classification1')